# MORPC Insights - Crash Injuries and Fatalities

## Overview

The Ohio Department of Transportation maintains a database of vehicle crash indidents, including the injuries and fatalities that sometimes result from them.  Each year, typically in the spring, MORPC receives an extract of this data that ODOT has cleaned.  This notebook produces a summary of injuries and fatalities based on the cleaned data.
  
This notebook produces a tileset that includes a summary of injuries and fatalities for 13 of the counties in the MORPC 15-county region and a selection of counties outside of this region.  The included counties in the MORPC region are Delaware*, Fairfield*, Franklin*, Knox*, Licking*, Madison*, Marion*, Morrow*, Perry, Pickaway*, Ross, Union*.  A summary is also produced for the MORPC 10-County Region, which includes the counties denoted with asterisks.

## Setup

### Load required libraries

In [ ]:
import pandas as pd
import frictionless
import requests
import os
import sys
import json
import datetime
import textwrap
import matplotlib
from matplotlib import pyplot as plt
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### User-specified parameters

In [ ]:
# Year range is the range of years that will be included in output tables that are indexed by year
YEAR_RANGE = [2019, 2023]

# Summary range is the range of years that will be included when computing summaries
SUMMARY_RANGE = [2019, 2023]

### Static parameters

In [ ]:
INPUT_DIR = os.path.normpath("./input_data")

OUTPUT_DIR = os.path.normpath("./output_data")

CHART_DIRNAME = "charts"

SELECTED_GEOS = ['Delaware', 'Fairfield', 'Franklin', 'Knox', 'Licking', 
                 'Madison', 'Marion', 'Morrow', 'Pickaway', 'Union', 'Region10']

# Dictionary mapping county codes to county names
COUNTY_CODE_MAP = {
    'CRA': 'Crawford',
    'DEL': 'Delaware',
    'FAI': 'Fairfield',
    'FRA': 'Franklin',
    'HAN': 'Hancock',
    'HAR': 'Hardin',
    'KNO': 'Knox',
    'LIC': 'Licking',
    'LOG': 'Logan',
    'MAD': 'Madison',
    'MAR': 'Marion',
    'MOT': 'Montgomery',
    'MRW': 'Morrow',
    'PER': 'Perry',
    'POR': 'Portage',
    'PIC': 'Pickaway',
    'RIC': 'Richland',
    'ROS': 'Ross',
    'SUM': 'Summit',
    'UNI': 'Union',
    'WYA': 'Wyandot'
}

OUTCOME_DESCRIPTION_MAP = {
    'INCAPAC_INJURIES_NBR':"Incapacitating injuries",
    'NON_INCAPAC_INJURIES_NBR':"Non-incapacitating injuries",
    'NO_INJURY_REPORTED_NBR':"No injuries",
    'ODPS_TOTAL_FATALITIES_NBR':"Fatality",
    'POSSIBLE_INJURIES_NBR':"Possible injuries"
}

### Define inputs

#### Create input data directory

Create input data directory if it doesn't exist.

In [ ]:
inputDir = os.path.normpath(INPUT_DIR)
if not os.path.exists(inputDir):
    os.makedirs(inputDir)

#### Cleaned crash incident records

As of March 2025, the crash data is relatively large (>70MB) and therefore is not well suited for storage in GitHub.  For lack of a better repository it is stored in ArcGIS Online.  A resource file and schema are captured in GitHub to allow for validation.

In [ ]:
CRASH_INPUT_URL = "https://www.arcgis.com/sharing/rest/content/items/97a9bde6b3b24358b52ca86964714f76/data"
CRASH_INPUT_TABLE_FILENAME = "MORPC_Crashes_2019-2023_GIS.csv"
CRASH_INPUT_TABLE_PATH = os.path.join(inputDir, CRASH_INPUT_TABLE_FILENAME)
CRASH_INPUT_RESOURCE_PATH = CRASH_INPUT_TABLE_PATH.replace(".csv",".resource.yaml")
CRASH_INPUT_SCHEMA_PATH = CRASH_INPUT_TABLE_PATH.replace(".csv",".schema.yaml")

### Define outputs

#### Create output data directory

Create output data directory if it doesn't exist.

In [ ]:
outputDir = os.path.normpath(OUTPUT_DIR)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)   

In [ ]:
chartDir = os.path.join(outputDir, CHART_DIRNAME)
if not os.path.exists(chartDir):
    os.makedirs(chartDir)    

#### Crashes by geo by year (all variables)

In [ ]:
OUTPUT_LONG_TABLE_FILENAME = "crashes-long.csv"
OUTPUT_LONG_TABLE_PATH = os.path.join(outputDir, OUTPUT_LONG_TABLE_FILENAME)
OUTPUT_LONG_SCHEMA_PATH = OUTPUT_LONG_TABLE_PATH.replace(".csv",".schema.yaml")
OUTPUT_LONG_RESOURCE_PATH = OUTPUT_LONG_TABLE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(OUTPUT_LONG_TABLE_PATH))
print("Schema: {}".format(OUTPUT_LONG_SCHEMA_PATH))
print("Resource file: {}".format(OUTPUT_LONG_RESOURCE_PATH))

#### Crash severity by geo by type

In [ ]:
OUTPUT_CRASH_SEVERITY_BY_TYPE_FILENAME = "crashSeverityByType.csv"
OUTPUT_CRASH_SEVERITY_BY_TYPE_PATH = os.path.join(outputDir, OUTPUT_CRASH_SEVERITY_BY_TYPE_FILENAME)
OUTPUT_CRASH_SEVERITY_BY_TYPE_SCHEMA = OUTPUT_CRASH_SEVERITY_BY_TYPE_PATH.replace(".csv",".schema.yaml")
OUTPUT_CRASH_SEVERITY_BY_TYPE_RESOURCE = OUTPUT_CRASH_SEVERITY_BY_TYPE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(OUTPUT_CRASH_SEVERITY_BY_TYPE_PATH))
print("Schema: {}".format(OUTPUT_CRASH_SEVERITY_BY_TYPE_SCHEMA))
print("Resource: {}".format(OUTPUT_CRASH_SEVERITY_BY_TYPE_RESOURCE))      

#### Crash severity by geo by year

In [ ]:
OUTPUT_CRASH_SEVERITY_BY_YEAR_FILENAME = "crashSeverityByYear.csv"
OUTPUT_CRASH_SEVERITY_BY_YEAR_PATH = os.path.join(outputDir, OUTPUT_CRASH_SEVERITY_BY_YEAR_FILENAME)
OUTPUT_CRASH_SEVERITY_BY_YEAR_SCHEMA = OUTPUT_CRASH_SEVERITY_BY_YEAR_PATH.replace(".csv",".schema.yaml")
OUTPUT_CRASH_SEVERITY_BY_YEAR_RESOURCE = OUTPUT_CRASH_SEVERITY_BY_YEAR_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(OUTPUT_CRASH_SEVERITY_BY_YEAR_PATH))
print("Schema: {}".format(OUTPUT_CRASH_SEVERITY_BY_YEAR_SCHEMA))
print("Resource: {}".format(OUTPUT_CRASH_SEVERITY_BY_YEAR_RESOURCE))      

#### Crash severity by geo by mode

In [ ]:
OUTPUT_CRASH_SEVERITY_BY_MODE_FILENAME = "crashSeverityByMode.csv"
OUTPUT_CRASH_SEVERITY_BY_MODE_PATH = os.path.join(outputDir, OUTPUT_CRASH_SEVERITY_BY_MODE_FILENAME)
OUTPUT_CRASH_SEVERITY_BY_MODE_SCHEMA = OUTPUT_CRASH_SEVERITY_BY_MODE_PATH.replace(".csv",".schema.yaml")
OUTPUT_CRASH_SEVERITY_BY_MODE_RESOURCE = OUTPUT_CRASH_SEVERITY_BY_MODE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(OUTPUT_CRASH_SEVERITY_BY_MODE_PATH))
print("Schema: {}".format(OUTPUT_CRASH_SEVERITY_BY_MODE_SCHEMA))
print("Resource: {}".format(OUTPUT_CRASH_SEVERITY_BY_MODE_RESOURCE))      

## Prepare input data

### Load cleaned crash incident records

Load incident records from ArcGIS online and save a local copy.  If a local copy exists, use that instead.

In [ ]:
if(os.path.exists(CRASH_INPUT_TABLE_PATH)):
    print("Using existing local copy of crash incident data found at {}".format(CRASH_INPUT_TABLE_PATH))
    crashesRaw = pd.read_csv(CRASH_INPUT_TABLE_PATH)
else:
    print("Downloading crash incident data from ArcGIS Online and saving a copy at {}".format(CRASH_INPUT_TABLE_PATH))
    crashesRaw = pd.read_csv(CRASH_INPUT_URL)
    crashesRaw.to_csv(CRASH_INPUT_TABLE_PATH, index=False)

Validate the crash incident dataset.

In [ ]:
valid = morpc.frictionless_validate_resource(CRASH_INPUT_RESOURCE_PATH)
if(not valid):
    print("Validation of crash incident data failed.  Inspect the data and ensure it matches the resource file and schema, then try again.")
    raise RuntimeError

Inspect the data.

In [ ]:
crashesRaw.head()

In [ ]:
crashesRaw["SEVERITY_BY_TYPE_CD"].value_counts()

In [ ]:
crashesRaw["CRASH_TYPE_CD"].value_counts()

In [ ]:
crashesRaw[['ODPS_TOTAL_FATALITIES_NBR','INCAPAC_INJURIES_NBR','NON_INCAPAC_INJURIES_NBR','POSSIBLE_INJURIES_NBR','NO_INJURY_REPORTED_NBR']].describe()

## Transform data to format required by Insights platform

### Simplify crash data

Create a working copy.

In [ ]:
crashes = crashesRaw.copy()

Convert three-letter county codes to county names.

In [ ]:
crashes['GEONAME'] = crashes['NLF_COUNTY_CD'].map(COUNTY_CODE_MAP)

Rename year column to make it match other columns.

In [ ]:
crashes = crashes.rename(columns={"Year":"YEAR"})

Extract only the records from the selected counties.

In [ ]:
crashes = crashes.loc[crashes["GEONAME"].isin(SELECTED_GEOS)].copy()

Remove the numeric prefix from the crash severity categories.

In [ ]:
crashes["SEVERITY_BY_TYPE_CD"] = crashes["SEVERITY_BY_TYPE_CD"].apply(lambda x:x.split(" ", maxsplit=1)[1])

Extract only the columns that we need.

In [ ]:
crashes = crashes.filter(items=['GEONAME','YEAR','SEVERITY_BY_TYPE_CD','CRASH_TYPE_CD'], axis="columns")

Summarize the crashes by county, year, crash type, and severity.

In [ ]:
crashes['CRASHES'] = 1
crashes = crashes \
    .groupby(['GEONAME','YEAR','CRASH_TYPE_CD','SEVERITY_BY_TYPE_CD']).sum() \
    .sort_values(['GEONAME','YEAR','CRASH_TYPE_CD','SEVERITY_BY_TYPE_CD']) \
    .reset_index()

Summarize the crashes for the 10-county region.

In [ ]:
temp = crashes.copy() \
    .drop(columns="GEONAME") \
    .groupby(['YEAR','CRASH_TYPE_CD','SEVERITY_BY_TYPE_CD']).sum() \
    .reset_index()
temp["GEONAME"] = "Region10"
crashes = pd.concat([crashes, temp], axis="index")

Inspect the data.

In [ ]:
crashes

In [ ]:
crashes[["GEONAME","YEAR","CRASHES"]].groupby(["GEONAME","YEAR"]).sum().reset_index().pivot(index="GEONAME",columns="YEAR",values="CRASHES")

### Summarize crash severity by geography by year

In [ ]:
crashSeverityByGeoByYear = crashes \
    .drop(columns=["CRASH_TYPE_CD"]) \
    .groupby(["GEONAME","YEAR","SEVERITY_BY_TYPE_CD"]).sum().reset_index() \
    .pivot(index=["GEONAME","YEAR"], columns=["SEVERITY_BY_TYPE_CD"], values="CRASHES") \
    .rename(columns={
        "Fatal":"GEO_YEAR_FATAL",
        "Injury Possible":"GEO_YEAR_POSSIBLE",
        "Minor Injury Suspected":"GEO_YEAR_MINOR",
        "PDO/No Injury":"GEO_YEAR_NOINJ",
        "Serious Injury Suspected":"GEO_YEAR_SERIOUS"
    }) \
    .filter(items=["GEO_YEAR_FATAL", "GEO_YEAR_SERIOUS", "GEO_YEAR_MINOR", "GEO_YEAR_POSSIBLE", "GEO_YEAR_NOINJ"], axis="columns") \

crashSeverityByGeoByYear["GEO_YEAR_TOTAL"] = crashSeverityByGeoByYear.sum(axis="columns")

crashSeverityByGeoByYear.columns.name = None
crashSeverityByGeoByYear.head()

In [ ]:
crashSeverityByGeoByYear.reset_index()[["GEONAME","YEAR","GEO_YEAR_FATAL"]].pivot(index="GEONAME",columns="YEAR",values="GEO_YEAR_FATAL")

In [ ]:
crashSeverityByGeoByYear.reset_index()[["GEONAME","YEAR","GEO_YEAR_SERIOUS"]].pivot(index="GEONAME",columns="YEAR",values="GEO_YEAR_SERIOUS")

### Summarize crash severity by geography for all years

In [ ]:
crashSeverityByGeo = crashSeverityByGeoByYear.copy() \
    .reset_index() \
    .drop(columns="YEAR") \
    .groupby("GEONAME").sum() \
    .rename(columns=(lambda column:column.replace("GEO_YEAR","GEO")))
crashSeverityByGeo

### Create table of crash severity by crash type

In [ ]:
crashSeverityByCrashType = crashes.loc[crashes["YEAR"].isin(range(SUMMARY_RANGE[0], SUMMARY_RANGE[1]+1))].copy() \
    .drop(columns="YEAR") \
    .groupby(["GEONAME","CRASH_TYPE_CD","SEVERITY_BY_TYPE_CD"]).sum() \
    .reset_index()

crashSeverityByCrashType = crashSeverityByCrashType.pivot(index=["GEONAME","CRASH_TYPE_CD"], columns="SEVERITY_BY_TYPE_CD", values="CRASHES") \
    .fillna(0) \
    .astype("int")

crashSeverityByCrashType.columns.name = None

crashSeverityByCrashType = crashSeverityByCrashType.rename(columns={
        "Fatal":"GEO_CRASHTYPE_FATAL",
        "Injury Possible":"GEO_CRASHTYPE_POSSIBLE",
        "Minor Injury Suspected":"GEO_CRASHTYPE_MINOR",
        "PDO/No Injury":"GEO_CRASHTYPE_NOINJ",
        "Serious Injury Suspected":"GEO_CRASHTYPE_SERIOUS"
    }) \
    .filter(items=["GEO_CRASHTYPE_FATAL", "GEO_CRASHTYPE_SERIOUS", "GEO_CRASHTYPE_MINOR", "GEO_CRASHTYPE_POSSIBLE", "GEO_CRASHTYPE_NOINJ"], axis="columns") \

crashSeverityByCrashType["GEO_CRASHTYPE_TOTAL"] = crashSeverityByCrashType.sum(axis="columns")

crashSeverityByCrashType = crashSeverityByCrashType.reset_index()                                                          

crashSeverityByCrashType = crashSeverityByCrashType \
    .merge(crashSeverityByGeo, on="GEONAME") \
    .set_index(["GEONAME","CRASH_TYPE_CD"])

crashSeverityByCrashType["GEO_CRASHTYPE_SHARE_OF_FATAL"] = crashSeverityByCrashType["GEO_CRASHTYPE_FATAL"] / crashSeverityByCrashType["GEO_FATAL"] 
crashSeverityByCrashType["GEO_CRASHTYPE_SHARE_OF_SERIOUS"] = crashSeverityByCrashType["GEO_CRASHTYPE_SERIOUS"] / crashSeverityByCrashType["GEO_SERIOUS"] 
crashSeverityByCrashType["GEO_CRASHTYPE_SHARE_OF_TOTAL"] = crashSeverityByCrashType["GEO_CRASHTYPE_TOTAL"] / crashSeverityByCrashType["GEO_TOTAL"] 

crashSeverityByCrashType = crashSeverityByCrashType.filter(items=[
    'GEO_CRASHTYPE_FATAL', 'GEO_CRASHTYPE_SERIOUS', 'GEO_CRASHTYPE_MINOR', 'GEO_CRASHTYPE_POSSIBLE', 'GEO_CRASHTYPE_NOINJ', 'GEO_CRASHTYPE_TOTAL',
    'GEO_CRASHTYPE_SHARE_OF_FATAL','GEO_CRASHTYPE_SHARE_OF_SERIOUS', 'GEO_CRASHTYPE_SHARE_OF_TOTAL'], axis="columns")

crashSeverityByCrashType.head()

In [ ]:
crashSeverityByCrashType.describe()

In [ ]:
crashSeverityByCrashType.loc["Region10"] \
    .filter(items=['GEO_CRASHTYPE_SHARE_OF_FATAL','GEO_CRASHTYPE_SHARE_OF_SERIOUS','GEO_CRASHTYPE_SHARE_OF_TOTAL'], axis="columns") \
    .sort_values("GEO_CRASHTYPE_SHARE_OF_FATAL", ascending=False) \
    .plot.barh(color=['red','orange','grey'], figsize=(10,10))

### Create table of crash severity by year

In [ ]:
crashSeverityByYear = crashSeverityByGeoByYear.copy()
crashSeverityByYear["GEO_YEAR_FATAL_RATE"] = crashSeverityByYear["GEO_YEAR_FATAL"] / (crashSeverityByYear["GEO_YEAR_TOTAL"] / 1000)
crashSeverityByYear["GEO_YEAR_SERIOUS_RATE"] = crashSeverityByYear["GEO_YEAR_SERIOUS"] / (crashSeverityByYear["GEO_YEAR_TOTAL"] / 1000)
crashSeverityByYear.head()

In [ ]:
temp = crashSeverityByYear.loc["Region10"].copy() \
    .reset_index() \
    .filter(items=["YEAR","GEO_YEAR_TOTAL","GEO_YEAR_FATAL_RATE","GEO_YEAR_SERIOUS_RATE"])
fig, ax1 = plt.subplots(figsize=(10, 7))
ax2 = ax1.twinx()
ax1.bar(temp["YEAR"], temp["GEO_YEAR_TOTAL"], color="grey", label="Total crashes")
ax2.plot(temp["YEAR"], temp['GEO_YEAR_FATAL_RATE'], color="red", label="Fatal crash rate")
ax2.plot(temp["YEAR"], temp['GEO_YEAR_SERIOUS_RATE'], color="orange", label="Serious injury crash rate")
yMax = temp[["GEO_YEAR_FATAL_RATE","GEO_YEAR_SERIOUS_RATE"]].max().max() * 1.1
ax2.set_ylim(ymin=0, ymax=yMax)
ax2.set_ylabel("Crash rate (per 1000 crashes)")
ax1.set_ylabel("Total crashes")
ax1Handles, ax1Labels = ax1.get_legend_handles_labels()
ax2Handles, ax2Labels = ax2.get_legend_handles_labels()
legend = fig.legend(ax1Handles+ax2Handles, ax1Labels+ax2Labels, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=3)


### Create table of crash severity by mode

In [ ]:
crashSeverityByMode = crashes.copy()
crashSeverityByMode["MODE"] = "Motor Vehicle"
crashSeverityByMode.loc[crashSeverityByMode["CRASH_TYPE_CD"] == "Pedalcycles", "MODE"] = "Pedalcycles"
crashSeverityByMode.loc[crashSeverityByMode["CRASH_TYPE_CD"] == "Pedestrian", "MODE"] = "Pedestrian"

crashSeverityByMode = crashSeverityByMode \
    .drop(columns=["YEAR","CRASH_TYPE_CD"]) \
    .groupby(["GEONAME","MODE","SEVERITY_BY_TYPE_CD"]).sum() \
    .reset_index() \
    .pivot(index=["GEONAME","MODE"], columns="SEVERITY_BY_TYPE_CD", values="CRASHES") \
    .rename(columns={
        "Fatal":"GEO_MODE_FATAL",
        "Injury Possible":"GEO_MODE_POSSIBLE",
        "Minor Injury Suspected":"GEO_MODE_MINOR",
        "PDO/No Injury":"GEO_MODE_NOINJ",
        "Serious Injury Suspected":"GEO_MODE_SERIOUS"
    }) \
    .fillna(0) \
    .astype("int")

crashSeverityByMode["GEO_MODE_TOTAL"] = crashSeverityByMode.sum(axis="columns")
crashSeverityByMode.columns.name = None

crashSeverityByMode = crashSeverityByMode.reset_index()                                                          

crashSeverityByMode = crashSeverityByMode \
    .merge(crashSeverityByGeo, on="GEONAME") \
    .set_index(["GEONAME","MODE"])

crashSeverityByMode.head()

In [ ]:
crashSeverityByMode["GEO_MODE_FATAL_LIKELIHOOD"] = crashSeverityByMode["GEO_MODE_FATAL"] / crashSeverityByMode["GEO_MODE_TOTAL"] 
crashSeverityByMode["GEO_MODE_SERIOUS_LIKELIHOOD"] = crashSeverityByMode["GEO_MODE_SERIOUS"] / crashSeverityByMode["GEO_MODE_TOTAL"] 
crashSeverityByMode["GEO_MODE_FATALSERIOUS_LIKELIHOOD"] = (crashSeverityByMode["GEO_MODE_FATAL"] + crashSeverityByMode["GEO_MODE_SERIOUS"]) / crashSeverityByMode["GEO_MODE_TOTAL"]
crashSeverityByMode["GEO_MODE_SHARE_OF_TOTAL"] = crashSeverityByMode["GEO_MODE_TOTAL"] / crashSeverityByMode["GEO_TOTAL"]
crashSeverityByMode.head()

In [ ]:
crashSeverityByMode.describe()

In [ ]:
crashSeverityByMode.loc["Region10"] \
    .reset_index() \
    .filter(items=['MODE','GEO_MODE_FATAL_LIKELIHOOD','GEO_MODE_SERIOUS_LIKELIHOOD','GEO_MODE_FATALSERIOUS_LIKELIHOOD','GEO_MODE_SHARE_OF_TOTAL'], axis="columns") \
    .set_index("MODE") \
    .plot.bar(color=["red","orange","orangered","grey"], figsize=(10,7))

## Export data and create resource files

### Crash severity by crash type

In [ ]:
outputCrashSeverityByCrashTypeSchema = morpc.frictionless_load_schema(OUTPUT_CRASH_SEVERITY_BY_TYPE_SCHEMA)

In [ ]:
outputCrashSeverityByCrashType = crashSeverityByCrashType.copy() \
    .filter(items=["GEO_CRASHTYPE_SHARE_OF_FATAL","GEO_CRASHTYPE_SHARE_OF_SERIOUS","GEO_CRASHTYPE_SHARE_OF_TOTAL"], axis="columns") \
    .reset_index() \
    .melt(id_vars=["GEONAME","CRASH_TYPE_CD"], var_name="VARIABLE", value_name="VALUE")

outputCrashSeverityByCrashType["GEONAME"] = outputCrashSeverityByCrashType["GEONAME"].apply(lambda name:("10-County Region" if name=="Region10" else "{} County".format(name)))

outputCrashSeverityByCrashType = morpc.cast_field_types(outputCrashSeverityByCrashType, outputCrashSeverityByCrashTypeSchema)

outputCrashSeverityByCrashType.head()

In [ ]:
outputCrashSeverityByCrashType.to_csv(OUTPUT_CRASH_SEVERITY_BY_TYPE_PATH, index=False)

In [ ]:
outputCrashSeverityByCrashTypeResource = morpc.frictionless_create_resource(OUTPUT_CRASH_SEVERITY_BY_TYPE_FILENAME, 
    resourcePath=OUTPUT_CRASH_SEVERITY_BY_TYPE_RESOURCE,
    title="MORPC Insights | Crash Severity by Crash Type", 
    name="crashes_severity_by_type", 
    description="Summary of vehicle crash severity by county by crash type for counties in the MORPC 10-county region. Variables include the share of total crashes, \
        accounted for by each type plus the share of fatal crashes and share of serious injury crashes.",
    writeResource=True,
    validate=True
)

### Crash severity by year

In [ ]:
outputCrashSeverityByYearSchema = morpc.frictionless_load_schema(OUTPUT_CRASH_SEVERITY_BY_YEAR_SCHEMA)

In [ ]:
outputCrashSeverityByYear = crashSeverityByYear.copy() \
    .filter(items=["GEO_YEAR_TOTAL","GEO_YEAR_FATAL_RATE","GEO_YEAR_SERIOUS_RATE"], axis="columns") \
    .reset_index() \
    .melt(id_vars=["GEONAME","YEAR"], var_name="VARIABLE", value_name="VALUE")

outputCrashSeverityByYear["GEONAME"] = outputCrashSeverityByYear["GEONAME"].apply(lambda name:("10-County Region" if name=="Region10" else "{} County".format(name)))

outputCrashSeverityByYear = morpc.cast_field_types(outputCrashSeverityByYear, outputCrashSeverityByYearSchema)

outputCrashSeverityByYear.head()

In [ ]:
outputCrashSeverityByYear.to_csv(OUTPUT_CRASH_SEVERITY_BY_YEAR_PATH, index=False)

In [ ]:
outputCrashSeverityByYearResource = morpc.frictionless_create_resource(OUTPUT_CRASH_SEVERITY_BY_YEAR_FILENAME, 
    resourcePath=OUTPUT_CRASH_SEVERITY_BY_YEAR_RESOURCE,
    title="MORPC Insights | Crash Severity by Year", 
    name="crashes_severity_by_year", 
    description="Summary of vehicle crash severity by county by year for counties in the MORPC 10-county region. Variables include the total number of crashes, \
        the rate of fatal crashes per 1000 crashes, and the rate of serious injury crashes per 1000 crashes.",
    writeResource=True,
    validate=True
)

### Crash severity by mode

In [ ]:
crashSeverityByMode.head()

In [ ]:
outputCrashSeverityByModeSchema = morpc.frictionless_load_schema(OUTPUT_CRASH_SEVERITY_BY_MODE_SCHEMA)

In [ ]:
outputCrashSeverityByMode = crashSeverityByMode.copy() \
    .filter(items=["GEO_MODE_FATAL_LIKELIHOOD","GEO_MODE_SERIOUS_LIKELIHOOD", "GEO_MODE_FATALSERIOUS_LIKELIHOOD", "GEO_MODE_SHARE_OF_TOTAL"], axis="columns") \
    .reset_index() \
    .melt(id_vars=["GEONAME","MODE"], var_name="VARIABLE", value_name="VALUE")

outputCrashSeverityByMode["GEONAME"] = outputCrashSeverityByMode["GEONAME"].apply(lambda name:("10-County Region" if name=="Region10" else "{} County".format(name)))

outputCrashSeverityByMode.head()

In [ ]:
outputCrashSeverityByMode.to_csv(OUTPUT_CRASH_SEVERITY_BY_MODE_PATH, index=False)

In [ ]:
outputCrashSeverityByModeResource = morpc.frictionless_create_resource(OUTPUT_CRASH_SEVERITY_BY_MODE_FILENAME, 
    resourcePath=OUTPUT_CRASH_SEVERITY_BY_MODE_RESOURCE,
    title="MORPC Insights | Crash Severity by Mode", 
    name="crashes_severity_by_mode", 
    description="Summary of vehicle crash severity by county by mode for counties in the MORPC 10-county region. Variables include the likelihood of a fatal crash, \
        likelihood of a serious injury crash, combined likelihood of a fatal or serious injury crash, and the share of total crashes that is accounted for by the mode.",
    writeResource=True,
    validate=True
)

## Generate static charts

In [ ]:
for f in os.scandir(chartDir):
    os.remove(f)

### Crash severity by crash type

In [ ]:
%matplotlib agg

colorset = ['#cc1414', '#f7b602', '#a8a8a8']
    
# Iterate over each geography in data set
for geoName in SELECTED_GEOS:   

    # Extract the data for a single geography
    geoLabel = ("10-County Region" if geoName == "Region10" else "{} County".format(geoName))

    # Extract the data for the geography and the columns required to be displayed on the chart. Sort the data
    # such that the crash types are ordered by share of fatal crashes
    temp = crashSeverityByCrashType.loc[geoName].copy() \
        .filter(items=['GEO_CRASHTYPE_SHARE_OF_FATAL','GEO_CRASHTYPE_SHARE_OF_SERIOUS','GEO_CRASHTYPE_SHARE_OF_TOTAL'], axis="columns") \
        .rename(columns={
            'GEO_CRASHTYPE_SHARE_OF_FATAL':"Share of fatal crashes",
            'GEO_CRASHTYPE_SHARE_OF_SERIOUS':"Share of serious injury crashes",
            'GEO_CRASHTYPE_SHARE_OF_TOTAL':"Share of all crashes"
        }) \
        .sort_values("Share of fatal crashes", ascending=False)

    # Convert the share ratio into a percentage
    for column in ['Share of fatal crashes','Share of serious injury crashes','Share of all crashes']:     
        temp[column] = temp[column] * 100

    # Generate a title string
    title = "Crash Severity by Type of Crash - {}".format(geoLabel)
    ylabel = "Percent of crashes"
    xlabel = None
    yLim = 1.1 * temp[['Share of fatal crashes','Share of serious injury crashes','Share of all crashes']].max().max()
      
    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))

    temp.plot.bar(ax=ax, color=colorset, width=0.8)

    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin=0, ymax=yLim)
    ax.set_yticks([round(tick,0) for tick in ax.get_yticks()])
    labels = [textwrap.fill(label, 15) for label in labels]
    legend = ax.legend(handles[::-1], labels[::-1], loc='upper right', labelspacing=1)
    ax.grid(visible=True, color="lightgrey", axis="y")
    ax.set_axisbelow(True)
       
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "crashSeverityByType-{}.svg".format(geoName)), bbox_extra_artists=(legend,), bbox_inches='tight')
    
    plt.close(ax.figure)

    writer = pd.ExcelWriter(os.path.join(chartDir, "crashSeverityByType-{}.xlsx".format(geoName)), engine='xlsxwriter')
    dataOptions = {
        "numberFormat": {
            'CRASH_TYPE_CD': "",
            'Share of fatal crashes': "0",
            'Share of serious injury crashes': "0",
            'Share of all crashes': "0",
        },
        "columnWidth": 32
    }
    chartOptions = {
        "colors": colorset,
        "titles": {
            "chartTitle": title,
            "xTitle": xlabel,
            "yTitle": ylabel
        },
        "xAxisOptions": {
            "num_font": {"size":12},
        },
        "yAxisOptions": {
            "num_font": {"size":14},
            "num_format": "0",
            "name_font": {"size":14}
        },
        "legendOptions":{
            "position":"bottom",
            "font":{"size":14}
        },
        "sizeOptions":{
            "x_scale":1.5,
            "y_scale":2
        }
    }
    morpc.data_chart_to_excel(temp, writer, chartType="column", dataOptions=dataOptions, chartOptions=chartOptions)
    writer.close()    
    
%matplotlib inline

### Crash severity by year

In [ ]:
%matplotlib agg

colorset = ['#cc1414', '#f7b602', '#a8a8a8']
   
# Iterate over each geography in data set
for geoName in SELECTED_GEOS:   

    # Extract the data for a single geography
    geoLabel = ("10-County Region" if geoName == "Region10" else "{} County".format(geoName))

    # Extract the data for the geography and the columns required to be displayed on the chart.
    temp = crashSeverityByYear.loc[geoName].copy() \
        .reset_index() \
        .filter(items=["YEAR","GEO_YEAR_TOTAL","GEO_YEAR_FATAL_RATE","GEO_YEAR_SERIOUS_RATE"]) \
        .rename(columns={
            "GEO_YEAR_TOTAL":"Total crashes",
            "GEO_YEAR_FATAL_RATE":"Fatal crash rate",
            "GEO_YEAR_SERIOUS_RATE":"Serious injury crash rate"
        })

    # Generate a title string
    title = "Crash Severity by Year - {}".format(geoLabel)
    yMax = temp[["Fatal crash rate","Serious injury crash rate"]].max().max() * 1.1

    lineWidth = 3
    markerSize = 7

    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax1 = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))
    ax2 = ax1.twinx()
    ax1.bar(temp["YEAR"], temp["Total crashes"], label="Total crashes", color=colorset[2])
    ax2.plot(temp["YEAR"], temp['Fatal crash rate'], label="Fatal crash rate", color=colorset[0], linewidth=lineWidth, marker='o', markersize=markerSize)
    ax2.plot(temp["YEAR"], temp['Serious injury crash rate'], label="Serious injury crash rate", color=colorset[1], linewidth=lineWidth, marker='o', markersize=markerSize)
    ax2.set_ylim(ymin=0, ymax=yMax)
    ax2.set_ylabel("Crash rate (per 1000 crashes)")
    ax1.set_ylabel("Total crashes")
    ax1Handles, ax1Labels = ax1.get_legend_handles_labels()
    ax2Handles, ax2Labels = ax2.get_legend_handles_labels()
    legend = fig.legend(ax1Handles+ax2Handles, ax1Labels+ax2Labels, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=3)
    ax1.set_title(title, fontsize=14)
    ax1.set_axisbelow(True)
    ax1.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
       
    # Save the figure to disk as an SVG file
    ax1.figure.savefig(os.path.join(chartDir, "crashSeverityByYear-{}.svg".format(geoName)), bbox_extra_artists=(legend,), bbox_inches='tight')
    
    plt.close(ax1.figure)

%matplotlib inline

### Crash severity by mode

In [ ]:
%matplotlib agg

colorset = ['#cc1414', '#f7b602', "#508efa", '#a8a8a8']
    
# Iterate over each geography in data set
for geoName in SELECTED_GEOS:   

    # Extract the data for a single geography
    geoLabel = ("10-County Region" if geoName == "Region10" else "{} County".format(geoName))

    # Extract the data for the geography and the columns required to be displayed on the chart. Sort the data
    # such that the crash types are ordered by share of fatal crashes
    temp = crashSeverityByMode.loc[geoName] \
        .reset_index() \
        .filter(items=['MODE','GEO_MODE_FATAL_LIKELIHOOD','GEO_MODE_SERIOUS_LIKELIHOOD','GEO_MODE_FATALSERIOUS_LIKELIHOOD','GEO_MODE_SHARE_OF_TOTAL'], axis="columns") \
        .rename(columns={
            'GEO_MODE_FATAL_LIKELIHOOD':"Percent of crashes resulting in fatality",
            'GEO_MODE_SERIOUS_LIKELIHOOD':"Percent of crashes resulting in serious injury",
            'GEO_MODE_FATALSERIOUS_LIKELIHOOD':"Percent of crashes resulting in serious injury or fatality",
            'GEO_MODE_SHARE_OF_TOTAL':"Share of all crashes"
        }) \
        .set_index("MODE")

    # Convert the share ratio into a percentage
    for column in ['Percent of crashes resulting in fatality','Percent of crashes resulting in serious injury','Percent of crashes resulting in serious injury or fatality','Share of all crashes']:     
        temp[column] = temp[column] * 100

    # Generate a title string
    title = "Crash Severity by Mode of Transportation - {}".format(geoLabel)
    ylabel = "Percent of crashes"
    xlabel = None
    #yLim = 1.1 * temp[['Percent of crashes resulting in fatality','Percent of crashes resulting in serious injury','Percent of crashes resulting in serious injury or fatality','Share of all crashes']].max().max()
    yLim = 100

    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))

    temp.plot.bar(ax=ax, color=colorset)

    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin=0, ymax=yLim)
    ax.set_yticks([round(tick,0) for tick in ax.get_yticks()])
    labels = [textwrap.fill(label, 15) for label in labels]
    legend = ax.legend(handles[::-1], labels[::-1], loc='upper right', labelspacing=1)
    ax.grid(visible=True, color="lightgrey", axis="y")
    ax.set_axisbelow(True)
       
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "crashSeverityByMode-{}.svg".format(geoName)), bbox_extra_artists=(legend,), bbox_inches='tight')
    
    plt.close(ax.figure)

%matplotlib inline

## Generate Insights catalog content

The content in the Insights platform is controlled by a catalog spreadsheet. Each tile to be displayed in the platform must have a record in the catalog.  This section will create the records for the tiles that display the alternative fuel station data.  Eventually this function will be performed by a separate staging script.

First specify the column names used in the catalog.

In [ ]:
columnNames=["TileID","TilesetID","Priority","GeoType","GeoName","Category","Headline","Commentary","ThumbnailURL","Contributor","Vintage","UpdateInterval","ShareURL","DataProductURL","MoreContextURL","TechDetailsURL"]

Create a new dataframe containing only the geographies for which thumbnail images were produced in the section above.

In [ ]:
catalog = pd.DataFrame(index=SELECTED_GEOS, columns=columnNames)
catalog.index.name = "geoLabel"
catalog = catalog.reset_index()

Convert the shortened geography labels to longer, more human-readable names.

In [ ]:
catalog["GeoName"] = catalog["geoLabel"].apply(lambda x:("10-County Region" if x == "Region10" else "{} County".format(x)))

Add the appropriate GeographyType values.

In [ ]:
catalog["GeoType"] = catalog["geoLabel"].apply(lambda x:("Region" if x == "Region10" else "County"))

Populate some placeholder fields.

In [ ]:
catalog["TileID"] = None
catalog["TilesetID"] = "TBD-crashSeverityByType"
catalog["Priority"] = None
catalog["Category"] = None
catalog["Headline"] = "TBD"
catalog["Commentary"] = "TBD"

Generate the URL for the thumbnail images. These will be hosted in GitHub and will be indexed by GEOIDFQ.

In [ ]:
catalog["ThumbnailURL"] = catalog["geoLabel"].apply(lambda name:"https://raw.githubusercontent.com/morpc-insights/insights-crashes/refs/heads/main/output_data/charts/crashSeverityByType-Delaware.svg".format(name))

Populate some other simple metadata.  Vintage in this case refers to the year that the content was published in Insights. This is to give readers an idea of how old it is.  UpdateInterval gives them an idea of when to expect the next version. ShareURL is a placeholder for now.

In [ ]:
catalog["Contributor"] = "Mid-Ohio Regional Planning Commission"
catalog["Vintage"] = str(datetime.date.today().year)
catalog["UpdateInterval"] = "annually"
catalog["ShareURL"] = None
catalog["DataProductURL"] = "https://www.arcgis.com/apps/dashboards/025b75f3490b4e79ae764e2c27c09a06"

Generate the URLs that point to the fact sheet.

In [ ]:
catalog["MoreContextURL"] = "https://morpc1-my.sharepoint.com/:w:/g/personal/aporr_morpc_org/EdF9OgInIZdOk9Sz3aJSVT8BPvXp4N495QUN2PvK5NBBjQ?e=b30KGA"

Generate the URLs that point to the data kit.

In [ ]:
catalog["TechDetailsURL"] = "https://github.com/morpc-insights/insights-crashes"

Extract only the required columns.

In [ ]:
catalog = catalog.filter(items=columnNames, axis="columns")

Inspect the listing.

In [ ]:
catalog.head()

Add records for the crash severity by year tileset, which are identical to the crash severity by type tileset but have different thumbnail URLs and data product URLs.

In [ ]:
template = catalog.copy()
temp = template.copy()
temp["ThumbnailURL"] = temp["ThumbnailURL"].str.replace("crashSeverityByType-", "crashSeverityByYear-")
temp["TilesetID"] = temp["TilesetID"].str.replace("crashSeverityByType", "crashSeverityByYear")
temp["DataProductURL"] = temp["DataProductURL"].str.replace("https://www.arcgis.com/apps/dashboards/025b75f3490b4e79ae764e2c27c09a06","https://www.arcgis.com/apps/dashboards/d1bb7ef5468f495788703352b1c5f896")
catalog = pd.concat([catalog, temp], axis="index")

Add records for the crash severity by mode tileset, which are identical to the crash severity by type tileset but have different thumbnail URLs and data product URLs.

In [ ]:
temp = template.copy()
temp["ThumbnailURL"] = temp["ThumbnailURL"].str.replace("crashSeverityByType-", "crashSeverityByMode-")
temp["TilesetID"] = temp["TilesetID"].str.replace("crashSeverityByType", "crashSeverityByMode")
temp["DataProductURL"] = temp["DataProductURL"].str.replace("https://www.arcgis.com/apps/dashboards/025b75f3490b4e79ae764e2c27c09a06","https://www.arcgis.com/apps/dashboards/2d3cc0a173d949f0a1a39146b37e1831")
catalog = pd.concat([catalog, temp], axis="index")

Save the catalog to an Excel spreadsheet.

In [ ]:
catalog.to_excel("catalog.xlsx", index=False)

It is necessary to manually add these records to the master catalog or update the records already therein.  See the following file in GitHub:

https://github.com/morpc/morpc-insights/blob/main/catalog/morpc_insights_catalog.xlsx
